# Hegselmann-Krause model: Simulation
I have written the following code for my essay on Modeling of Opinion Dynamics as part of my degree at the University of Cambridge (Part III of the Mathematical Tripos).

In [ ]:
using Plots
using Random  
using LaTeXStrings
using Distributions
using LinearAlgebra

import Measures: Length, AbsoluteLength, Measure, BoundingBox, mm, cm, pt, width, height, w, h

const plt = Plots
const rnd = Random
const LA = LinearAlgebra

## Simulation functions
The following functions lie at the core of the HK simulation and either simulate full trajectories in opinion space or simulate the equilibrium state (until the system has converged)

### Auxiliary functions

In [ ]:
# One time step in the HK model
# for symmetric intervals call with ϵ_l = ϵ_r
# x_i is the initial state, x_f is the updated state, err_tol specifies how exact the agreement with the confidence bound has to be
# noise_function determines the noise added (see functions in "Noise functions") with parameter noise_param
function HK_update!(x_i, x_f, ϵ_l, ϵ_r; err_tol = 1e-9, noise_function = no_jumps, noise_param = 0.)
    #x_i and x_f have to be distinct vectors and not pointers to the same entity
    N =  convert(Int,length(x_i))
    
    #Calculate interaction matrix
    I = Matrix{Float64}(LA.I, N, N)
    for i in 1:(N-1)
        for j in (i+1):N
            # if i (smaller) lies within the left bound of j (bigger), then i influences j
            if abs(x_i[i]-x_i[j]) < ϵ_l+err_tol
                I[j,i] = 1.0
            else
            end
            # if j (bigger) lies within the right bound of i (smaller), then j influences i
            if abs(x_i[i]-x_i[j]) < ϵ_r+err_tol
                I[i,j] = 1.0
            else
            end
        end
    end
    #normalize each row of the row-stochastic matrix
    for i in 1:N
        norm = sum(I[i,:])
        I[i,:] = I[i,:]./norm
    end
    
    #Generate noise if required
    ξ = noise_function(noise_param, N)


    #Update state vector
    for i in 1:N
        x_update = 0.

        # average over neigbouring opinions
        for j in 1:N
            if I[i,j]>err_tol
                x_update += I[i,j] * x_i[j] 
            else
            end
        end

        # add noise
        x_update += + ξ[i]
        
        # Adsorbing boundary condition
        if x_update > 1
            x_update = 1
        elseif x_update < 0
            x_update = 0
        else
        end
        
        #update final state vector
        x_f[i] = round(x_update, digits = 9)
    end
end


# Update state for state dependent confidence bounds 
# only difference to above in calculation of I matrix (ϵ)
# with min, mid and max as described for epsilon_quadratic (see "Variable confidence bound functions")
function HK_update_variable!(x_i, x_f, min, mid, max; err_tol = 1e-9, noise_function = no_jumps, noise_param = 0.)
    #x_i and x_f have to be distinct vectors and not pointers to the same entity
    N =  convert(Int,length(x_i))

    #Calculate interaction matrix
    I = Matrix{Float64}(LA.I, N, N)
    
    for i in 1:(N-1)
        for j in (i+1):N
            # if i (smaller) lies within the left bound of j (bigger), then i influences j
            if abs(x_i[i]-x_i[j]) < epsilon_quadratic(x_i[j], min, mid, max)[1] +err_tol ### CHANGE
                I[j,i] = 1.0
            else
            end
            # if j (bigger) lies within the right bound of i (smaller), then j influences i
            if abs(x_i[i]-x_i[j]) < epsilon_quadratic(x_i[i], min, mid, max)[2]+err_tol ### CHANGE
                I[i,j] = 1.0
            else
            end
        end
    end
    #normalize each row of the row-stochastic matrix
    for i in 1:N
        norm = sum(I[i,:])
        I[i,:] = I[i,:]./norm
    end
    
    #Generate noise if required
    ξ = noise_function(noise_param, N) 


    #Update state vector
    for i in 1:N
        x_update = 0.

        # average over neigbouring opinions
        for j in 1:N
            if I[i,j]>err_tol
                x_update += I[i,j] * x_i[j] 
            else
            end
        end

        # add noise
        x_update += + ξ[i]
        
        # Adsorbing boundary condition
        if x_update > 1
            x_update = 1
        elseif x_update < 0
            x_update = 0
        else
        end
        
        #update final state vector
        x_f[i] = round(x_update, digits = 9)
    end
end


# a function checking if two vectors a and be have converged up to a tolerance err_tol
function check_convergence(a::Vector,b::Vector; err_tol = 1e-9)
    if length(a) != length(b)
        println("Not same length")
        return -1
    else
        null = zeros(length(a)) .+ err_tol

        change = abs.(a .- b)
        check = change .< null
        if check == ones(length(check))
            return 1
        else
            return 0
        end
    end
end


# a function to check if a state has converged and if not give a hint why
function convergence(a,b; err_tol = 1e-9)
    if check_convergence(a,b) != 1
        println("Not converged!")
    
        letzt = (unique(b))
        vorletzt = (unique(a)) 
        if length(letzt) == length(vorletzt)
            plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=200, legend = :none)
            plt.plot!(xlabel = "Opinions", ylabel = "Change")
            plt.scatter!(vorletzt, letzt.-vorletzt)
        else
            print("Merging of clusters")
        end
        return -1
    else
        return 1
    end
end

### Simulation runs

In [ ]:
# simulate a trajectory in opinion space 
# for left and right confidence bounds ϵ_l, ϵ_r, N agents, an initial distribution initial, 
# a function specifying how the noise is dsitributed noise_function with a parameter noise_param
# and a random seed for uniformly dsitributed initial conditions seed
function HK_simulate(ϵ_l, ϵ_r;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0., seed = 2310)
    #Initial Distribution
    rng = MersenneTwister(seed)
    if initial == "rand"
        x_i = rand!(rng, zeros(N))
        sort!(x_i)#, rev=:true)
    elseif initial == "uni"
        x_i = collect(0:1/(N-1):1)
    elseif initial == "const"
        x_i = ones(N) .* 0.5
    else
        println("Choose either rand or uni or const for intial distribution")
        return -1
    end

    #Initialize Simulation
    trajectories = zeros(N,max_iter+1)
    trajectories[:,1] = x_i
    x_old = x_i
    x_new = copy(x_i)
    
    #Simulation
    for i in 1:max_iter
        HK_update!(x_old,x_new, ϵ_l, ϵ_r; noise_function = noise_function, noise_param = noise_param)
        trajectories[:,i+1] = x_new
        x_old = copy(x_new)
    end
    return trajectories
    
end


# simulate as above, just for symmetric confidence bounds ϵ = ϵ_l = ϵ_r
function HK_simulate(ϵ;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0., seed= 2310)
    trajectories = HK_simulate(ϵ, ϵ; N = N, max_iter = max_iter, initial = initial, noise_function = noise_function, noise_param = noise_param, seed= seed)
    return trajectories
end


# Simulation loop for specified initial condition array x_i
function HK_simulate(x_i::Array, ϵ_l, ϵ_r; max_iter = 20, noise_function = no_jumps, noise_param = 0., seed = 2310)

    #Initialize Simulation
    trajectories = zeros(length(x_i),max_iter+1)
    trajectories[:,1] = x_i
    x_old = x_i
    x_new = copy(x_i)
    
    #Simulation
    for i in 1:max_iter
        HK_update!(x_old,x_new, ϵ_l, ϵ_r; noise_function = noise_function, noise_param = noise_param)
        trajectories[:,i+1] = x_new
        x_old = copy(x_new)
    end
    return trajectories
    
end


# Simulate trajectories for variable confidence bounds
# only difference is update function called (HK_update_variable!())
# with min, mid and max as described for epsilon_quadratic (see "Variable confidence bound functions")
function HK_simulate_variable(min, mid, max;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0.)
    #Initial Distribution
    rng = MersenneTwister(2310)
    if initial == "rand"
        x_i = rand!(rng, zeros(N))
        sort!(x_i)#, rev=:true)
    elseif initial == "uni"
        x_i = collect(0:1/(N-1):1)
    elseif initial == "const"
        x_i = ones(N) .* 0.5
    else
        println("Choose either rand or uni or const for intial distribution")
        return -1
    end

    #Initialize Simulation
    trajectories = zeros(N,max_iter+1)
    trajectories[:,1] = x_i
    x_old = x_i
    x_new = copy(x_i)
    
    #Simulation
    for i in 1:max_iter
        HK_update_variable!(x_old,x_new, min, mid, max; noise_function = noise_function, noise_param = noise_param) ### CHANGE
        trajectories[:,i+1] = x_new
        x_old = copy(x_new)
    end
    return trajectories
end


# a simulation is run until the states have converged (equilibrium state reached) and the last two simulations states returned 
# input works the same as in HK_simulate
function HK_simulate_convergence(ϵ_l, ϵ_r;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0., seed = 2310)
    #Initial Distribution
    rng = MersenneTwister(seed)
    if initial == "rand"
        x_i = rand!(rng, zeros(N))
        sort!(x_i)#, rev=:true)
    elseif initial == "uni"
        x_i = collect(0:1/(N-1):1)
    elseif initial == "const"
        x_i = ones(N) .* 0.5
    else
        println("Choose either rand or uni or const for intial distribution")
        return -1
    end

    #Initialize Simulation
    x_old = x_i
    x_new = copy(x_i)
    
    #Simulation
    for k in 1:max_iter
        HK_update!(x_old,x_new, ϵ_l, ϵ_r; noise_function = noise_function, noise_param = noise_param)

        if check_convergence(x_old, x_new; err_tol = 1e-9) == 1
            #print("Converged at ", k)
            break
        elseif k != max_iter
            x_old = copy(x_new)
        else
        end
    end
    convergence(x_old, x_new; err_tol = 1e-9) != 1 ? println("not converged within max_iter") :
    return x_old,x_new
end

## Noise functions
These functions can be used to simulate the noisy HK model with differently distributed noise $\xi$

In [ ]:
# creates no noise, i.e. a noise vector of zeros
# the paraneter "param" is just to fit the overall structure of noise functions
function no_jumps(param, N)
    return zeros(N)    
end


# creates zero-mean Gaussian random noise with stddev "strength"
function gaussian_random_jumps(strength, N)
    μ = 0
    σ = strength
    d = Normal(μ, σ) 
    ξ = rand(d,N)
    return ξ
end


# creates uniformly distributed noise up to strength "bound", i.e. ξ ∈ [-bound, bound]
function uniform_bounded_jumps(bound, N)
    d = Uniform(-bound,bound)
    ξ = rand(d,N)
    return ξ
end

## Variable confidence bound functions
The following functions implement the state dependent confidence bound described in the essay and only work for an opinion space $\mathcal{O} = [0,1]$

In [ ]:
# This function produces the amplifying quadratic function in the essay for an opinion x
# the parameter min is the minimum of this curve (in the center) and max the maximum (at the extremes x=0 or 1)
function epsilon_quadratic_amplify(x, min, max)
    epsilon = min + 4* (x-0.5)^2 * (max-min)
    return round(epsilon, digits = 9)
end


# This function produces the attenuating part of the confidence bound accordingly
function epsilon_quadratic_attenuate(x, min, max; middle = 0.5)
    epsilon = - 4 * (max-min) * (x-0.5)^2 + max
    return round(epsilon, digits = 9)
end


# This function combines the previous two functions to amplify the extremes that agree with an agents own opinion x 
# min is the lowest possible confidence param (l), max the maximum (h) and mid the center where attenuating and aplification meet (m)
function epsilon_quadratic(x::Number, min, mid, max; middle = 0.5)
    
    if x > middle
        ϵ_l = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
        ϵ_r = epsilon_quadratic_amplify(x, mid, max; middle = middle)
    else
        ϵ_l = epsilon_quadratic_amplify(x, mid, max; middle = middle)
        ϵ_r = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
    end

    return [ϵ_l, ϵ_r]
end


# Same idea as previous method of epsilon_quadratic, just for a state vector
function epsilon_quadratic(x_vec::Vector, min, mid, max; middle = 0.5)
    ϵ_left = zeros(length(x_vec))
    ϵ_right = zeros(length(x_vec))

    for (i,x) in enumerate(x_vec)
        if x > middle
            ϵ_l = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
            ϵ_r = epsilon_quadratic_amplify(x, mid, max; middle = middle)
        else
            ϵ_l = epsilon_quadratic_amplify(x, mid, max; middle = middle)
            ϵ_r = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
        end
        
        ϵ_left[i] = ϵ_l 
        ϵ_right[i] = ϵ_r
    end
    return [ϵ_left, ϵ_right ]
end

## Example analysis
A short example how the functions can be used to produce opinion trajectories

In [ ]:
#specify parameters
N = 50
ϵ = 0.20
max_iter = 30
initial = "uni"

#run simulation
result = HK_simulate(ϵ; N = N, max_iter = max_iter, initial = initial, seed = 2310)

#plotting result of simulation for max_iter simulation steps
max_iter = 10
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
for i in 1:N
    plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
end

plt.plot!()